In [7]:
from typing import Callable
import math

# Bisection Method

In [24]:
def bisection_with_check(f: Callable[[float], float], a: float, b:float, tol=1e-8, max_iter=100):
    # --- 1. Tecken-testet (Säkerhetsspärren) ---
    val_a = f(a)
    val_b = f(b)
    
    # Om f(a) och f(b) har samma tecken (produkten blir positiv), avbryt.
    if val_a * val_b > 0:
        print(f"VARNING: Funktionen byter inte tecken mellan {a} och {b}.")
        print(f"f({a}) = {val_a:.4f}")
        print(f"f({b}) = {val_b:.4f}")
        print("Metoden kräver att f(a) och f(b) har olika tecken.")
        return None
    
    print(f"Test OK! Funktionen byter tecken. Roten är instängd.")
    print(f"{'Iter':<5} {'a':<12} {'b':<12} {'Mittpunkt (c)':<15} {'f(c)':<12}")
    print("-" * 60)

    # --- 2. Iterationen ---
    for k in range(max_iter):
        c = (a + b) / 2  # Hitta mitten
        val_c = f(c)
        
        print(f"{k+1:<5} {a:<12.6f} {b:<12.6f} {c:<15.6f} {val_c:<12.6f}")

        if abs(val_c) < tol or (b - a) / 2 < tol:
            print(f"\nKonvergerade! Roten är ca {c:.6f}")
            return c
        
        # Avgör vilken halva vi ska spara
        # Om f(a) och f(c) har olika tecken ligger roten i vänstra halvan [a, c]
        if f(a) * val_c < 0:
            b = c
        else:
            a = c
            
    print("Konvergerade inte inom max antal iterationer.")
    return c

# Newton Raphson Method

In [28]:
def newton_raphson_with_check(f: Callable[[float],float], f_prime: Callable[[float],float], x0, tol=1e-8, max_iter=100):
    # --- 1. Derivata-testet (Startpunkten) ---
    # Vi kollar så att vi inte startar på en platt yta (derivata = 0)
    deriv = f_prime(x0)
    if abs(deriv) < 1e-10:
        print("VARNING: Derivatan är nära noll redan vid start.")
        print("Välj en annan startgissning för att undvika division med noll.")
        return None

    print(f"Startar Newton-Raphson vid x0 = {x0}")
    print(f"{'Iter':<5} {'x_nuvarande':<15} {'f(x)':<15} {'f\'(x)':<15} {'Nästa steg (dx)'}")
    print("-" * 75)

    x = x0
    for k in range(max_iter):
        val = f(x)
        deriv = f_prime(x)

        # Säkerhetskoll inuti loopen: Blev derivatan 0 plötsligt?
        if abs(deriv) < 1e-10:
            print(f"Avbryter: Derivatan blev för nära noll vid x={x:.4f}")
            return None

        # Newton-steget: dx = f(x) / f'(x)
        dx = val / deriv
        x_new = x - dx

        print(f"{k+1:<5} {x:<15.6f} {val:<15.6f} {deriv:<15.6f} {-dx:<15.6f}")

        if abs(dx) < tol:
            print(f"\nKonvergerade! Roten är ca {x_new:.8f}")
            return x_new
        
        x = x_new

    print("Konvergerade inte.")
    return x

# Fixed Point Iteration Method

In [30]:
def fixed_point_with_check(g, x0, tol=1e-6, max_iter=100):
    # --- 1. Konvergens-testet (Derivatan) ---
    h = 1e-6
    try:
        # Vi approximerar derivatan: (g(x+h) - g(x)) / h
        # Vi tar absolutbeloppet eftersom tecknet inte spelar roll för konvergens, bara lutningen.
        deriv_approx = abs((g(x0 + h) - g(x0)) / h)
    except ValueError:
        print("VARNING: Kunde inte beräkna derivatan (hamnade utanför definitionsmängden).")
        return None

    print(f"Startar Fixpunkt vid x0 = {x0}")
    print(f"Uppskattad derivata |g'(x0)| ≈ {deriv_approx:.4f}")

    if deriv_approx >= 1:
        print("VARNING: Derivatan är >= 1. Metoden kommer troligen DIVERGERA (sticka iväg).")
        print("Försök skriva om ekvationen så att x hamnar under en rot eller logaritm.")
        # Vi fortsätter ändå för att visa vad som händer, men varnar först.
    else:
        print("Test OK! Derivatan är < 1. Metoden bör KONVERGERA.")

    print("-" * 65)
    print(f"{'Iter':<5} {'x_gammal':<15} {'x_ny (g(x))':<15} {'Förändring':<15}")
    print("-" * 65)

    # --- 2. Iterationen ---
    x_old = x0
    for k in range(max_iter):
        try:
            x_new = g(x_old)
        except ValueError:
            print(f"Krasch! Försökte beräkna g({x_old:.4f}) men det gick inte (t.ex. rot ur negativt tal).")
            return None

        change = abs(x_new - x_old)
        
        print(f"{k+1:<5} {x_old:<15.6f} {x_new:<15.6f} {change:<15.6f}")

        if change < tol:
            print(f"\nKonvergerade! Roten är ca {x_new:.6f}")
            return x_new
        
        x_old = x_new

    print("Konvergerade inte inom max antal iterationer.")
    return x_old

# Testing the methods 

In [34]:
# NEWTON-RAPHSON = Hitta nollpunkt!
# Exempel: x^2 - 2 = 0 (Vi vill hitta roten ur 2)
# f(x) = x^2 - 2
# f'(x) = 2x
f_newton = lambda x: x**2 - 2
print(f"Newton Raphson for {str(f_newton)}")
df_newton = lambda x: 2*x

# Startgissning 1.0
root = newton_raphson_with_check(f_newton, df_newton, 1.0)
print("\n" * 5)


# BISECTION EXEMPEL = Hitta nollpunkt!
# Exempel: x^3 - 2x - 5 = 0
# Vi gissar intervallet [2, 3]
f_ex = lambda x: x**3 - 2*x - 5
print(f"Bisection for {str(f_ex)}")
root = bisection_with_check(f_ex, 2, 3)
print("\n" * 5)


# FIXED POINT ITERATION = Hitta nollpunkt!
# Ekvation: e^x + x = 7
# Vi vill hitta x där e^x + x - 7 = 0.
# DÅLIGT VAL: x = 7 - e^x
# (e^x växer fort, derivatan blir stor -> Divergens)
print(f"Fixed point iteration")
def g_bad(x):
    return 7 - math.exp(x)

# BRA VAL: x = ln(7 - x)
# (ln plattar till tal, derivatan blir liten -> Konvergens)
def g_good(x):
    return math.log(7 - x)

print("--- TEST 1: Dålig omskrivning ---")
fixed_point_with_check(g_bad, 1.0)
print("\n--- TEST 2: Bra omskrivning ---")
fixed_point_with_check(g_good, 1.0)


Newton Raphson for <function <lambda> at 0x729b54a90360>
Startar Newton-Raphson vid x0 = 1.0
Iter  x_nuvarande     f(x)            f'(x)           Nästa steg (dx)
---------------------------------------------------------------------------
1     1.000000        -1.000000       2.000000        0.500000       
2     1.500000        0.250000        3.000000        -0.083333      
3     1.416667        0.006944        2.833333        -0.002451      
4     1.414216        0.000006        2.828431        -0.000002      
5     1.414214        0.000000        2.828427        -0.000000      

Konvergerade! Roten är ca 1.41421356






Bisection for <function <lambda> at 0x729b54aa4f40>
Test OK! Funktionen byter tecken. Roten är instängd.
Iter  a            b            Mittpunkt (c)   f(c)        
------------------------------------------------------------
1     2.000000     3.000000     2.500000        5.625000    
2     2.000000     2.500000     2.250000        1.890625    
3     2.000000    

1.6728216638780904